In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
## read the data set:
df=pd.read_excel('data/Online Retail.xlsx')

## Data Exploration

In [19]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [8]:
df.shape

(541909, 8)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [12]:
df.describe().T.round(2)

,count,mean,min,25%,50%,75%,max,std
Quantity,541909.0,9.55225,-80995.0,1.0,3.0,10.0,80995.0,218.081158
InvoiceDate,541909,2011-07-04 13:34:57.156386048,2010-12-01 08:26:00,2011-03-28 11:34:00,2011-07-19 17:17:00,2011-10-19 11:27:00,2011-12-09 12:50:00,NaN
UnitPrice,541909.0,4.611114,-11062.06,1.25,2.08,4.13,38970.0,96.759853
CustomerID,406829.0,15287.69057,12346.0,13953.0,15152.0,16791.0,18287.0,1713.600303


In [16]:
## dropp null values
df.dropna(inplace=True)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      406829 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.9+ MB


In [33]:
df.groupby('Description').agg({'Quantity':'sum'}).sort_values('Quantity',ascending=False).head()

,Quantity
Description,
WORLD WAR 2 GLIDERS ASSTD DESIGNS,53215
JUMBO BAG RED RETROSPOT,45066
ASSORTED COLOUR BIRD ORNAMENT,35314
WHITE HANGING HEART T-LIGHT HOLDER,34147
PACK OF 72 RETROSPOT CAKE CASES,33409


In [43]:
## Number of canceled Order
df['InvoiceNo'].str.contains('C').count()

8905

In [46]:
## remove canceled order
df=df[~df['InvoiceNo'].str.contains('C',na=False)]

In [48]:
df.shape

(397924, 8)

In [58]:
df['Total_price']=df['Quantity']* df['UnitPrice']

C:\Users\ThinkPad\AppData\Local\Temp\ipykernel_4596\3171869601.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Total_price']=df['Quantity']* df['UnitPrice']


In [62]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total_price
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


## RFM Analycis

In [66]:
import datetime as dt

In [72]:
df['InvoiceDate'].max()

Timestamp('2011-12-09 12:50:00')

In [74]:
today_time=dt.datetime(2011,12,11)

In [76]:
today_time

datetime.datetime(2011, 12, 11, 0, 0)

In [82]:
rfm=df.groupby('CustomerID').agg(
    {
        'InvoiceDate':lambda date:(today_time-date.max()).days,
        'InvoiceNo':lambda num:num.nunique(),
        'Total_price':lambda total:total.sum()
    }
)
rfm

,InvoiceDate,InvoiceNo,Total_price
CustomerID,,,
12346.0,326,1,77183.60
12347.0,3,7,4310.00
12348.0,76,4,1797.24
12349.0,19,1,1757.55
12350.0,311,1,334.40
...,...,...,...
18280.0,278,1,180.60
18281.0,181,1,80.82
18282.0,8,2,178.05


In [84]:
## raname collumns name
rfm.columns=['Recency','Frequency','Monetary']
rfm

,Recency,Frequency,Monetary
CustomerID,,,
12346.0,326,1,77183.60
12347.0,3,7,4310.00
12348.0,76,4,1797.24
12349.0,19,1,1757.55
12350.0,311,1,334.40
...,...,...,...
18280.0,278,1,180.60
18281.0,181,1,80.82
18282.0,8,2,178.05


In [90]:
rfm=rfm[rfm['Monetary']>0]

In [92]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,4338.0,93.059474,100.012264,1.00,18.000,51.000,142.75,374.00
Frequency,4338.0,4.272706,7.706221,1.00,1.000,2.000,5.00,210.00
Monetary,4338.0,2054.266460,8989.230441,3.75,307.415,674.485,1661.74,280206.02


In [100]:
rfm['Recency_score']=pd.qcut(rfm['Recency'],5,labels=[5,4,3,2,1])
rfm['Frequency_score']=pd.qcut(rfm['Frequency'].rank(method='first'),5,labels=[5,4,3,2,1])
rfm['Monetary_score']=pd.qcut(rfm['Monetary'],5,labels=[5,4,3,2,1])

C:\Users\ThinkPad\AppData\Local\Temp\ipykernel_4596\2202485889.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm['Recency_score']=pd.qcut(rfm['Recency'],5,labels=[5,4,3,2,1])
C:\Users\ThinkPad\AppData\Local\Temp\ipykernel_4596\2202485889.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm['Frequency_score']=pd.qcut(rfm['Frequency'].rank(method='first'),5,labels=[5,4,3,2,1])
C:\Users\ThinkPad\AppData\Local\Temp\ipykernel_4596\2202485889.py:3: SettingWithCopyWarning: 
A value is trying to be set on 

In [102]:
rfm

,Recency,Frequency,Monetary,Recency_score,Frequency_score,Monetary_score
CustomerID,,,,,,
12346.0,326,1,77183.60,1,5,1
12347.0,3,7,4310.00,5,1,1
12348.0,76,4,1797.24,2,2,2
12349.0,19,1,1757.55,4,5,2
12350.0,311,1,334.40,1,5,4
...,...,...,...,...,...,...
18280.0,278,1,180.60,1,4,5
18281.0,181,1,80.82,1,4,5
18282.0,8,2,178.05,5,3,5


In [104]:
rfm['Total']=rfm['Recency_score'].astype(str)+rfm['Frequency_score'].astype(str)

C:\Users\ThinkPad\AppData\Local\Temp\ipykernel_4596\1263518487.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm['Total']=rfm['Recency_score'].astype(str)+rfm['Frequency_score'].astype(str)


In [112]:
rfm

,Recency,Frequency,Monetary,Recency_score,Frequency_score,Monetary_score,Total
CustomerID,,,,,,,
12346.0,326,1,77183.60,1,5,1,15
12347.0,3,7,4310.00,5,1,1,51
12348.0,76,4,1797.24,2,2,2,22
12349.0,19,1,1757.55,4,5,2,45
12350.0,311,1,334.40,1,5,4,15
...,...,...,...,...,...,...,...
18280.0,278,1,180.60,1,4,5,14
18281.0,181,1,80.82,1,4,5,14
18282.0,8,2,178.05,5,3,5,53


In [126]:
# Segmenting Customers Using RFM Score
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'[4-5][4-5]': 'champions'
}
rfm['segment'] = rfm['Total'].replace(seg_map, regex=True)
rfm.head()


C:\Users\ThinkPad\AppData\Local\Temp\ipykernel_4596\3208201464.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfm['segment'] = rfm['Total'].replace(seg_map, regex=True)


,Recency,Frequency,Monetary,Recency_score,Frequency_score,Monetary_score,Total,segment
CustomerID,,,,,,,,
12346.0,326,1,77183.60,1,5,1,15,cant_loose
12347.0,3,7,4310.00,5,1,1,51,new_customers
12348.0,76,4,1797.24,2,2,2,22,hibernating
12349.0,19,1,1757.55,4,5,2,45,loyal_customers
12350.0,311,1,334.40,1,5,4,15,cant_loose


In [128]:
rfm.sample(10)

,Recency,Frequency,Monetary,Recency_score,Frequency_score,Monetary_score,Total,segment
CustomerID,,,,,,,,
13780.0,8,2,360.01,5,4,4,54,champions
18250.0,311,1,349.27,1,4,4,14,at_Risk
13085.0,158,2,416.08,2,4,4,24,at_Risk
17673.0,2,6,1629.12,5,1,2,51,new_customers
12902.0,265,1,138.68,1,5,5,15,cant_loose
17420.0,51,3,598.83,3,2,3,32,about_to_sleep
12358.0,2,2,1168.06,5,4,2,54,champions
15227.0,37,5,1219.40,3,2,2,32,about_to_sleep
12818.0,262,1,1542.08,1,5,2,15,cant_loose


In [130]:
rfm.groupby('segment')[['Recency', 'Frequency', 'Monetary']].agg(['mean', 'count', 'max']).round(2)

Recency            Frequency            Monetary        \
                       mean count  max      mean count  max     mean count   
segment                                                                      
about_to_sleep        49.97   324   72      5.72   324   63  2467.36   324   
at_Risk              192.06   879  374      1.67   879    3   684.11   879   
cant_loose           221.92   565  374      1.00   565    1   515.69   565   
champions              7.63    99   13      1.32    99    2   502.99    99   
hibernating          135.77   283  373      4.88   283   34  1589.84   283   
loyal_customers       42.24   565   72      1.18   565    2   450.04   565   
need_attention        52.43   187   72      2.33   187    3   897.63   187   
new_customers          6.20   439   13     16.05   439  210  9204.02   439   
potential_loyalists   16.18   752   33      3.39   752    6  1457.47   752   
promising             22.71   245   33      9.97   245   51  4738.17   245   

                                
                           max  
segment                         
about_to_sleep        81024.84  
at_Risk               44534.30  
cant_loose            77183.60  
champions              3861.00  
hibernating           10254.18  
loyal_customers        6207.67  
need_attention        12601.83  
new_customers        280206.02  
potential_loyalists  168472.50  
promising            124914.53